<a href="https://colab.research.google.com/github/LabuBu-Toy/Machine-Learning-68/blob/main/ML_Assignment__standard_team_lastest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fetal-health-classification 👶🏻🐣🍼

Dataset Link: https://www.kaggle.com/datasets/andrewmvd/fetal-health-classification


In [ ]:
!pip install imblearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from pprint import pprint
from fnmatch import fnmatchcase

In [ ]:
import warnings
warnings.filterwarnings('ignore') # Ignore any warning

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report,  log_loss,
    confusion_matrix, ConfusionMatrixDisplay, recall_score,
    balanced_accuracy_score, make_scorer, precision_score,
    f1_score, roc_auc_score, average_precision_score)

from imblearn.metrics import classification_report_imbalanced
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE, SMOTEN, SMOTENC, KMeansSMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import TomekLinks

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
from sklearn.base import clone
from fnmatch import fnmatchcase
from collections import Counter, defaultdict
from math import ceil
from sklearn.feature_selection import SelectKBest, f_classif


In [ ]:
from dataclasses import dataclass, field
from typing import Any, Optional, Iterable
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV

from imblearn.over_sampling import (
    RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE,
    SMOTEN, SMOTENC, KMeansSMOTE
)
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier, RUSBoostClassifier

In [ ]:
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.decomposition import PCA

In [ ]:
sns.set(style='darkgrid')

In [ ]:
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, balanced_accuracy_score, confusion_matrix
from scipy.stats import randint, loguniform

In [ ]:
import kagglehub
from pathlib import Path

# Download Dataset from Kaggle
dataset_path = kagglehub.dataset_download("andrewmvd/fetal-health-classification")

file_path = Path(dataset_path)
file_path = file_path / 'fetal_health.csv'

In [ ]:
# Set Random SEED
SEED = 42

In [ ]:
data = pd.read_csv(file_path)
display(data)
data.info()

## Data Cleaning & preparation





In [ ]:

print("ขนาดก่อนลบ:", data.shape)
data = data.drop_duplicates(keep="first")
print("ขนาดหลังลบ:", data.shape)


### Handling Missing Values

In [ ]:
pprint(f"Data Shape: {data.shape}")
pprint(f"Missing values: {data.isnull().sum().sum()}")

In [ ]:
# Percentage per class
pprint(data['fetal_health'].value_counts(normalize=True, sort=True)
     .mul(100).round(4))

In [ ]:
# Class distribution
plt.figure(figsize=(8,5))
sns.countplot(x='fetal_health', data=data, palette='Greens')
plt.title('Fetal Health Class Distribution')
plt.xlabel('Class (1: Normal, 2: Suspect, 3: Pathological)')
plt.ylabel('Count')
plt.show()

In [ ]:
# Counts per class (sorted by feature label)
pprint(data['fetal_health'].value_counts(sort=True))

In [ ]:
# Percentage per class
pprint(data['fetal_health'].value_counts(normalize=True, sort=True)
     .mul(100).round(4))


> Dataset are Imbalance

#### When if data imbalance What will goes wrong

- Skewed decision boundary: Model predicts the majority most of the time → low recall for minority.

- Misleading metrics: Overall accuracy/micro-F1 look high even if minority recall ≈ 0. ROC-AUC can look okay; PR (precision–recall) is more honest.

- Unstable training: Few minority examples → high variance, overfitting (especially if you duplicate them).

- Poor calibration: Probabilities get biased toward the majority prior.

> Tiny example (990 zeros, 10 ones): always predicting “0” gives 99% accuracy, but minority recall = 0%.


> **Baseline accuracy:** the share of the majority class. If class 1 is 70%, a naive "always‑1" model gets 70% accuracy —why accuracy alone can mislead.

---

### What to do if it’s class imbalanced

* **Metrics:** use **F1 (macro/weighted)**, **recall** for critical classes (e.g., Pathological), **PR‑AUC**.
* **Class weights:**  Pass Class weights to models that support `class_weight`.





* **Resampling:** oversample minority (e.g., **SMOTE**) or undersample majority (use carefully to avoid losing signal).
* **Stratify splits:** Strategic
* **Threshold tuning:** Adjust decision thresholds to trade precision vs recall per class.

### Common pitfalls

* **Accuracy paradox:** High accuracy from predicting only the majority class.
* **Data leakage:** Balancing after splitting can leak information—apply resampling **within** cross‑validation folds.
* **Over‑synthetic patterns:** Aggressive SMOTE settings can overfit; monitor with validation curves.

---

> TL-DR: Class distribution = "**How many (or what percent) of each label do we have?**"
Check it early. If it’s skewed, choose proper metrics, stratify, consider class weights/resampling, and tune thresholds.

## Distribution of Key Variables

We analyzed histograms to understand the distribution of critical variables and their relationships with the target variable.

In [ ]:
# Get all feature columns except the target variable
feature_columns = data.columns.drop('fetal_health')

# Plot histograms for all feature columns, colored by fetal_health
for feature in feature_columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=data, x=feature, hue='fetal_health', multiple='stack', kde=True, palette='viridis')
    plt.title(f'Distribution of {feature} by Fetal Health')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.show()

## Data Correlation Heatmap

A correlation heatmap is a color-coded grid that shows the pairwise correlation coefficients between variables in a dataset. Each row/column is a variable; each cell’s color and number indicate how strongly two variables move together:

Value range: from –1 to +1

- +1: perfect positive linear relationship (as one increases, the other increases)

- 0: no linear relationship

- –1: perfect negative linear relationship (as one increases, the other decreases)

> Symmetry: the matrix is symmetric; the diagonal is always 1.

In [ ]:
# Correlation heatmap pearson method
plt.figure(figsize=(15, 12))
sns.heatmap(data.corr(method="pearson"), annot=True, fmt='.2f', cmap="coolwarm", square=True) #use
plt.title('Feature Correlation Heatmap pearson method')
plt.show()

In [ ]:
# Correlation heatmap Spearman
plt.figure(figsize=(15, 12))
sns.heatmap(data.corr(method="spearman"), annot=True, fmt='.2f', cmap="coolwarm", square=True) #use
plt.title('Feature Correlation Heatmap spearman method')
plt.show()

# Step 3: Data Prepareation

Separate features and target variable.**bold text**

In [ ]:
# Features / Target
X = data.drop(columns=['fetal_health']).select_dtypes(include=[np.number]).copy()   # explicit list + copy for safety

# Optional: drop all columns starting with "histogram" (case-insensitive)
# X = X.loc[:, ~X.columns.str.lower().str.startswith('histogram')]

# Kaggle labels are floats (1.0/2.0/3.0); casting to int is often nicer
y = data['fetal_health'].astype('int64')

# 2) Check Data Shape
print(X.shape, y.shape)

assert 'fetal_health' not in X.columns
# assert not any(X.columns.str.lower().str.startswith('histogram'))  # if you dropped them

# Step 4: Data Train-Test-Split and k-Fold Cross Validation





In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,          # preserves class proportions in each split
    random_state=SEED
)

# Step 5: Models Training, Tuning and Evaluation


## Configuration

In [ ]:
# ======== CONFIG ========
smote_like = (SMOTE, BorderlineSMOTE, SVMSMOTE, SMOTEN, SMOTENC, KMeansSMOTE, SMOTEENN, SMOTETomek, TomekLinks)

PRIMARY_SCORER = "f1_macro"

In [ ]:
# Metrics:  Accuracy, Precision, Recall, plus your F1s
# NOTE: log-loss scorer in sklearn is named "neg_log_loss" (maximized),
# so we expose it as "log_loss" and flip the sign when reporting.
scoring = {
    "f1_macro": "f1_macro",
    "f1_weighted": "f1_weighted",
    "accuracy": "accuracy",
    "balanced_acc": "balanced_accuracy",
    "precision_macro": "precision_macro",
    "precision_weighted": "precision_weighted",
    "recall_macro": "recall_macro",
    "recall_weighted": "recall_weighted",

}


## Function Helper

Define a helper function for training and evaluating models.

In [ ]:
def grid_for(model_name: str, grid_map: dict) -> dict | None:
    # allow both exact and wildcard matches; also fix "LogReg_*" -> "LogReg*"
    grid_map = {k.replace("_*", "*"): v for k, v in grid_map.items()}
    if model_name in grid_map:
        return grid_map[model_name]
    for pattern, grid in grid_map.items():
        if fnmatchcase(model_name, pattern):
            return grid
    return None

def find_smote_param_prefixes(estimator):
    """Return param prefixes pointing to SMOTE-like instances (supports nesting, e.g. 'sampler__smote')."""
    prefixes = []
    for k, v in estimator.get_params(deep=True).items():
        if isinstance(v, smote_like):
            prefixes.append(k)
    return prefixes

def safe_k_neighbors(y, n_splits, default_k=5):
    """Safe SMOTE k per worst-case train fold."""
    counts = Counter(y)
    min_train = min(c - ceil(c / n_splits) for c in counts.values())
    if min_train < 2:
        return None
    return max(1, min(default_k, min_train - 1))

def pipeline_has_smote_or_smoteenn(estimator) -> bool:
    """Return True if any parameter in the estimator tree is an instance of a SMOTE-family sampler or SMOTEENN."""
    for _, v in estimator.get_params(deep=True).items():
        if isinstance(v, smote_like):
            return True
    return False

def evaluate_on_test(est, X_test = X_test, y_test = y_test):
    """Return dict of test metrics and show confusion matrices."""

    y_pred = est.predict(X_test)
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "balanced_acc": balanced_accuracy_score(y_test, y_pred),
        "precision_macro": precision_score(y_test, y_pred, average="macro", zero_division=0),
        "precision_weighted": precision_score(y_test, y_pred, average="weighted", zero_division=0),
        "recall_macro": recall_score(y_test, y_pred, average="macro", zero_division=0),
        "recall_weighted": recall_score(y_test, y_pred, average="weighted", zero_division=0),
        "f1_macro": f1_score(y_test, y_pred, average="macro"),
        "f1_weighted": f1_score(y_test, y_pred, average="weighted"),
    }
    # log loss & ROC-AUC if proba/decision available
    if hasattr(est, "predict_proba"):
        y_prob = est.predict_proba(X_test)
        try:
            metrics["log_loss"] = log_loss(y_test, y_prob, labels=np.unique(y_test))
        except Exception:
            pass
        try:
            metrics["roc_auc_ovr"] = roc_auc_score(y_test, y_prob, multi_class="ovr")
        except Exception:
            pass
    print("#" * 20)

    print("\n- Imbalance Classification report (test):")
    print(classification_report_imbalanced(y_test, y_pred, digits=4, zero_division=0))

    # Confusion matrices
    try:
        ConfusionMatrixDisplay.from_estimator(est, X_test, y_test)
        plt.title("Confusion Matrix (counts)")
        plt.show()
        ConfusionMatrixDisplay.from_estimator(est, X_test, y_test, normalize="true")
        plt.title("Confusion Matrix (normalized)")
        plt.show()
    except Exception:
        pass

    print("#" * 20)
    return metrics

In [ ]:
pipelines = {
    "RF_balancedRF": ImbPipeline([
        ("sel", SelectKBest(score_func=f_classif)), # supervised selection
        ("clf", BalancedRandomForestClassifier(
            bootstrap=True,
            random_state=SEED
        ))
    ]),
    "RF_pure" : ImbPipeline([
      ("clf", RandomForestClassifier(
          bootstrap=True,
          oob_score=False,
          random_state=SEED))
    ]),
    "RF_class_weighted" : ImbPipeline([
      ("clf", RandomForestClassifier(
          bootstrap=True,
          oob_score=False,
          class_weight="balanced_subsample",
          random_state=SEED))
    ]),
    "RF_FS": ImbPipeline([
        ("sel", SelectKBest(score_func=f_classif)), # supervised selection
        ("clf", RandomForestClassifier(
            bootstrap=True,
            class_weight="balanced_subsample",
            random_state=SEED
        ))
    ]),
    "RF_BorderlineSMOTE": ImbPipeline([
        ("scale", StandardScaler()),
        ("sel", SelectKBest(score_func=f_classif)), # supervised selection
        ("borderline_smote", BorderlineSMOTE(random_state=SEED)),
        ("clf", RandomForestClassifier(
            bootstrap=True,
            random_state=SEED))
    ]),

    "RF_SMOTE": ImbPipeline([
        ("scale", StandardScaler()),
        ("sel", SelectKBest(score_func=f_classif)), # supervised selection
        ("smote", SMOTE(random_state=SEED)),
        ("clf", RandomForestClassifier(
            bootstrap=True,
            random_state=SEED))
    ]),
    "RF_SMOTETomek": ImbPipeline([
        ("scale", StandardScaler()),
        ("sel", SelectKBest(score_func=f_classif)), # supervised selection
        ("smtomek", SMOTETomek(random_state=SEED)),
        ("clf", RandomForestClassifier(
            bootstrap=True,
            random_state=SEED))
    ])
}

## Hyperparameter tuning

In [ ]:
from scipy.stats import randint, loguniform, uniform

search_spaces = {
    # === Baselines ===
    "RF_balancedRF": {
        "sel": [SelectKBest(score_func=f_classif, k=10), SelectKBest(score_func=f_classif, k=20)],
        "clf__n_estimators": [300, 600, 1000],
        "clf__max_depth": [None, 10, 20, 30],
        "clf__min_samples_leaf": [1, 2, 4],
        "clf__max_features": ["sqrt", 0.5, 0.8],
        "clf__n_jobs": [-1],
    },
    "RF_pure": {
        "clf__n_estimators": [300, 600, 1000],
        "clf__max_depth": [None, 10, 20, 30],
        "clf__min_samples_leaf": [1, 2, 4],
        "clf__max_features": ["sqrt", 0.5, 0.8],
        "clf__n_jobs": [-1],
    },
    "RF_class_weighted": {
        "sel": [SelectKBest(score_func=f_classif, k=10), SelectKBest(score_func=f_classif, k=20)],
        "clf__n_estimators": [300, 600, 1000],
        "clf__max_depth": [None, 10, 20, 30],
        "clf__min_samples_leaf": [1, 2, 4],
        "clf__max_features": ["sqrt", 0.5, 0.8],
        "clf__n_jobs": [-1],
    },
    "RF_FS" : {
        "sel": [SelectKBest(score_func=f_classif), SelectKBest(score_func=f_classif, k=20)],
        "clf__n_estimators": [300, 600, 1000],
        "clf__max_depth": [None, 10, 20, 30],
        "clf__min_samples_leaf": [1, 2, 4],
        "clf__max_features": ["sqrt", 0.5, 0.8],
        "clf__n_jobs": [-1],
    },

    # === Samplers ===
    "RF_SMOTE": {
        "sel": [SelectKBest(score_func=f_classif, k=10), SelectKBest(score_func=f_classif, k=20)],
        "smote__k_neighbors": [3, 5, 7],                                   # SMOTE uses k-NN
        "smote__sampling_strategy": ["not majority", "all"],               # strings for multiclass
        "clf__n_estimators": [300, 600, 1000],
        "clf__max_depth": [None, 10, 20],
        "clf__min_samples_leaf": [1, 2, 4],
        "clf__max_features": ["sqrt", 0.5],
        "clf__n_jobs": [-1],
    },
    "RF_BorderlineSMOTE": {
        "sel": [SelectKBest(score_func=f_classif, k=10), SelectKBest(score_func=f_classif, k=20)],
        "borderline_smote__k_neighbors": [3, 5],
        "borderline_smote__m_neighbors": [5, 10],
        "borderline_smote__kind": ["borderline-1", "borderline-2"],
        "clf__n_estimators": [300, 600, 1000],
        "clf__max_depth": [None, 10, 20],
        "clf__min_samples_leaf": [1, 2, 4],
        "clf__max_features": ["sqrt", 0.5],
        "clf__n_jobs": [-1],
    },
    "RF_SMOTETomek": {
        "sel": [SelectKBest(score_func=f_classif, k=10), SelectKBest(score_func=f_classif, k=20)],
        "smtomek__smote__k_neighbors": [3, 5],                             # nested estimator
        "smtomek__sampling_strategy": ["not majority", "all"],
        "clf__n_estimators": [300, 600, 1000],
        "clf__max_depth": [None, 10, 20],
        "clf__min_samples_leaf": [1, 2, 4],
        "clf__max_features": ["sqrt", 0.5],
        "clf__n_jobs": [-1],
    }
}

## Cross Validation Process





In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=SEED)

## Skip tune model that use SMOTE and their families

In [ ]:
# --- build the filtered dict + a small report ---
skipped, pipelines_to_tune = [], {}
for name, pipe in pipelines.items():
    if pipeline_has_smote_or_smoteenn(pipe):
        skipped.append(name)
    else:
        pipelines_to_tune[name] = pipe

## Tuning Process

In [ ]:
print("[TUNE SELECTED] Will tune these pipelines: ", end="\t")
pprint(pipelines_to_tune.keys())

if len(skipped) > 0:
    print("[TUNE SKIPED] Skipping pipelines using SMOTE-Families: ", end="\t")
    pprint(skipped)

In [70]:
n_splits = getattr(cv, "n_splits", None) or cv.get_n_splits(X_train, y_train)
print(f"[CV] n_splits={n_splits}")

model_tuned: dict[str, dict] = defaultdict(dict)

for model_name, base_est in pipelines_to_tune.items():
    grid = grid_for(model_name, search_spaces)
    if not grid:
        print(f"[SKIP] {model_name}: no matching search space.")
        continue

    est = clone(base_est)

    print(f"[TUNE] {model_name} with params: {list(grid.keys()) or '[no tunable params]'}")

    search = HalvingRandomSearchCV(
        estimator=est,
        param_distributions=grid,
        scoring=PRIMARY_SCORER,
        cv=cv,
        factor=3,
        random_state=SEED,
        n_jobs=-1,
        error_score=np.nan,   # don't bomb entire run on a single bad fold
        refit="f1_macro",
        verbose=1,
    )

    # Perform the hyperparameter search by fitting the search object
    search.fit(X_train, y_train)

    # Store the tuning results and the best estimator directly
    model_tuned[model_name]["tune_results"] = {
        "best_score": float(search.best_score_),
        "best_params": search.best_params_,
        "n_candidates": getattr(search, "n_candidates_", None),
        "n_iterations": getattr(search, "n_iterations_", None),
    }
    model_tuned[model_name]["tuned_estimators"] = search.best_estimator_

[CV] n_splits=100
[TUNE] RF_balancedRF with params: ['sel', 'clf__n_estimators', 'clf__max_depth', 'clf__min_samples_leaf', 'clf__max_features', 'clf__n_jobs']
n_iterations: 1
n_required_iterations: 1
n_possible_iterations: 1
min_resources_: 600
max_resources_: 1690
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 2
n_resources: 600
Fitting 100 folds for each of 2 candidates, totalling 200 fits


KeyboardInterrupt: 

# Report Models

## Summary Tune Table

In [ ]:
# Summary table
summary_df = (
    pd.DataFrame([
        {"model": name, **vals["tune_results"]} for name, vals in model_tuned.items()
    ]).sort_values("best_score", ascending=False).reset_index(drop=True)
)
# Rename the 'best_score' column to be more explicit
summary_df.rename(columns={"best_score": "best_f1_macro_CV_score"}, inplace=True)

display(summary_df)

## Imbalance Classification report of Tuned Model

In [ ]:
# Evaluate all tuned models on the test set
print("Evaluating all tuned models on the test set:")

tuned_models_list = [name for name, vals in model_tuned.items() if "best_score" in vals.get("tune_results", {})]

if not tuned_models_list:
    print("No tuned models found in model_tune dictionary.")
else:
    for model_name in tuned_models_list:
        print(f"\nEvaluating tuned model: {model_name}")
        # Access the fitted tuned estimator
        tuned_estimator = model_tuned[model_name]["tuned_estimators"]

        # Evaluate on the test set
        test_metrics = evaluate_on_test(tuned_estimator, X_test, y_test)

        # Display scores for the tuned model in a table
        print(f"\nSummary of Scores for Tuned Model: {model_name} on Test Set:")
        tuned_scores_df = pd.DataFrame([test_metrics])
        display(tuned_scores_df)

## Train untuned model

In [ ]:
# Evaluate all original, untuned models on the test set for comparison
print("Evaluating original, untuned models:")
print(model_tuned.keys())

for name, pipeline in pipelines.items():
    if (name in pipelines_to_tune.keys()): # Check if the model is in the pipelines_to_tune list
      print(f"\nSkipping already tuned model: {name}")
      continue
    print(f"\nEvaluating untuned model: {name}")
    # Fit the original pipeline on the entire training data before evaluating on test set
    #pipeline.set_params(**best_params_for_train_untuned_model_dict)
    pipeline.fit(X_train, y_train)

    model_tuned[name]["tuned_estimators"] = pipeline
    model_tuned[name]["tune_results"] = evaluate_on_test(pipeline, X_test, y_test)

# Step 6: Model evaluation


## Ranking for each ML models

In [ ]:
# Create a list of dictionaries for the comparison DataFrame
comparison_data = []
display_metrics_order = [
    "f1_macro", "f1_weighted", "balanced_acc", "average_precision_macro",       # ใช้คัดโมเดล
    "recall_macro", "recall_weighted", "precision_macro", "precision_weighted"  # วินิจฉัยสมดุล P/R
    "mcc",                                                   # สมดุลรวม
    "roc_auc_ovr", "roc_auc_ovr_macro", "log_loss",          # อ้างอิง/โพรบา
    "accuracy"                                               # รายงานประกอบ
]

for name, vals in model_tuned.items():
    row = {"model": name}
    print(f"model: ", name)
    # Check if 'tune_results' contains tuning results or test metrics
    if "best_score" in vals["tune_results"]:
        # This is a tuned model, get test metrics from evaluate_on_test
        test_metrics = evaluate_on_test(vals["tuned_estimators"], X_test, y_test)
        row.update(test_metrics)
        row["model"] = f"{name} (Tuned)" # Add (Tuned) to the name
    else:
        # This is an original model, test metrics are already in tune_results
        print("|-> This is an original model, test metrics are already in tune_results")
        row.update(vals["tune_results"])
        row["model"] = f"{name} (Original)" # Add (Original) to the name

    comparison_data.append(row)

In [ ]:
# Select and reorder columns for display
# Create the comparison DataFrame
comparison_df = pd.DataFrame(comparison_data)

# Select and reorder columns for display
comparison_df = comparison_df[['model'] + [m for m in display_metrics_order if m in comparison_df.columns]].sort_values("f1_macro", ascending=False).reset_index(drop=True)

# Sort by f1_macro, a metric suitable for class imbalance
display(comparison_df)

# Select the best model based on the highest f1_macro score
best_model_name = comparison_df.loc[0, "model"]
print(f"\nBest Model based on f1_macro: {best_model_name}")

pprint(model_tuned[best_model_name.split()[0]]["tuned_estimators"])

## Export Models

In [ ]:
import joblib
import os

# Get the best model name from the comparison DataFrame
best_model_name_key = best_model_name.split()[0] # Get the original model name key

# Retrieve the best estimator from the model_tuned dictionary
best_estimator = model_tuned[best_model_name_key]["tuned_estimators"]

# Define the filename for saving the model
model_filename = f"{best_model_name_key}_model.joblib"

# Save the best estimator to the file
joblib.dump(best_estimator, model_filename)

print(f"Best model '{best_model_name_key}' saved to '{model_filename}'")
print("\nModel Estimators: ")
pprint(best_estimator)

# Optional: Verify the file exists
if os.path.exists(model_filename):
    print(f"\nFile '{model_filename}' successfully created.")